## 상품정보 데이터

### json데이터 불러오기

In [36]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## items

In [ ]:
items = pd.read_json('./items.json')
items.head()

In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [45]:
review = pd.read_json('./reviews.json')
review.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17997 entries, 0 to 17996
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   품번                     17997 non-null  int64  
 1   아이디                    17997 non-null  object 
 2   별점                     17997 non-null  int64  
 3   키                      13605 non-null  object 
 4   몸무게                    12570 non-null  object 
 5   평소사이즈                  12863 non-null  object 
 6   타입                     17451 non-null  object 
 7   리뷰                     17997 non-null  object 
 8   크림(cream)              1 non-null      object 
 9   사이즈                    274 non-null    object 
 10  컬러                     66 non-null     object 
 11  size                   5 non-null      float64
 12  color                  5 non-null      object 
 13  기장 타입                  71 non-null     object 
 14  색상-사이즈                 175 non-null    object 
 15  옵션

In [46]:
# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈','size', '컬러', '색상-사이즈', '색상']] # '데님', '타입2'

In [6]:
review.head()

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,size,컬러,색상-사이즈,색상
0,550,핫핑-연재,5,166~170,51 ~ 55,M,None,안사면 손해.. 완전 대만족~!\n베이직한 니트 찾으시는 분들\n교복템 여기있어요 ...,None,None,None,None,None
1,550,핫핑-민지,5,156~160,46 ~ 50,55,None,멋스러운 반팔니트\n가을부터 겨울까지 활용 가능한 아이템은 반팔니트가 아닐까 싶어요...,F,None,라일락그레이,None,None
2,550,150567241,5,171~175,65~,77,크림(cream)-F,바지랑입기 딱좋을거 같아서 구매 했는데 너무 예뻐요~\n바지를 더 구매하고싶은 욕구...,None,None,None,None,None
3,550,298224934,5,156~160,46 ~ 50,55,크림(cream)-F,이런 니트반팔 사고싶었는데 진짜 이뻐요 사이즈도 잘 맞아요 얼른 입고 나가고 싶어요,None,None,None,None,None
4,550,kies0,5,156~160,56 ~ 60,66,크림(cream)-F,도톰하니. 맘에들어요\n천도 관리만잘하면 보풀도 잘일어나지않을것같아요\n여름끝자락~...,None,None,None,None,None


In [47]:
# NaN -> None
review = review.replace({np.nan: None})
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17997 entries, 0 to 17996
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17997 non-null  int64 
 1   아이디     17997 non-null  object
 2   별점      17997 non-null  int64 
 3   키       13605 non-null  object
 4   몸무게     12570 non-null  object
 5   평소사이즈   12863 non-null  object
 6   타입      17451 non-null  object
 7   리뷰      17997 non-null  object
 8   사이즈     274 non-null    object
 9   size    5 non-null      object
 10  컬러      66 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      38 non-null     object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


### 카디널리티 확인

In [ ]:
for col in ['별점', '키', '몸무게', '평소사이즈']:
    print(col)
    print(review[col].unique())
    print('\n')

* 핫핑 사이즈 기준

<img src='./imgs/사이즈통일.png'>

In [48]:
# 평소 사이즈 수정
key1 = [str(i) for i in range(44, 100, 11)] +['100']
key2 = ['S', 'M', 'L', 'XL', 'XXL', 'XXXL']

def user_size(row):
    for i in range(1,6):
        if row == key1[i]:
            return str(i)
        elif row == key2[i]:
            return str(i)
        
review['평소사이즈'] = review['평소사이즈'].apply(user_size)
review['평소사이즈'].unique()

array(['1', '3', '2', None, '4', '5'], dtype=object)

In [49]:
# 리뷰 데이터 줄바꿈 -> 공백으로 대체
def review_comm(row):
    return re.sub('\n', ' ', row)

review['리뷰'] = review['리뷰'].apply(review_comm)
review.head()

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,size,컬러,색상-사이즈,색상
0,550,핫핑-연재,5,166~170,51 ~ 55,1,None,안사면 손해.. 완전 대만족~! 베이직한 니트 찾으시는 분들 교복템 여기있어요 🙋‍...,None,None,None,None,None
1,550,핫핑-민지,5,156~160,46 ~ 50,1,None,멋스러운 반팔니트 가을부터 겨울까지 활용 가능한 아이템은 반팔니트가 아닐까 싶어요!...,F,None,라일락그레이,None,None
2,550,150567241,5,171~175,65~,3,크림(cream)-F,바지랑입기 딱좋을거 같아서 구매 했는데 너무 예뻐요~ 바지를 더 구매하고싶은 욕구가...,None,None,None,None,None
3,550,298224934,5,156~160,46 ~ 50,1,크림(cream)-F,이런 니트반팔 사고싶었는데 진짜 이뻐요 사이즈도 잘 맞아요 얼른 입고 나가고 싶어요,None,None,None,None,None
4,550,kies0,5,156~160,56 ~ 60,2,크림(cream)-F,도톰하니. 맘에들어요 천도 관리만잘하면 보풀도 잘일어나지않을것같아요 여름끝자락~가을...,None,None,None,None,None


### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [ ]:
# review.loc[review['사이즈']=='핑크']

In [ ]:
# # 값 서로 바꾸기
# idx = review.loc[review['사이즈']=='핑크'].index
# review.loc[idx, '사이즈'], review.loc[idx, '컬러'] = review.loc[idx, '컬러'], review.loc[idx, '사이즈']
# review.iloc[idx, :]

In [ ]:
# 카디널리티 확인
cols = ['타입', '사이즈','size', '컬러', '색상-사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [50]:
idx = review.loc[review['사이즈']=='15'].index
review.drop(idx, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   타입      17451 non-null  object
 7   리뷰      17996 non-null  object
 8   사이즈     273 non-null    object
 9   size    5 non-null      object
 10  컬러      65 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      38 non-null     object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


#### 사이즈 있는경우 '색상-사이즈', 없는 경우 '색상' 형태로 값 통일

In [51]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free' or row=='프리':
        return 'F'
    elif row=='M':
        return '2'
    
review['사이즈'] = review['사이즈'].apply(size)

review.loc[review['사이즈']=='블랙 1', '사이즈'] = '블랙-1'

# 컬러 컬럼
def color(row):
    if row: # Nan => None
        if '_' in row:
            return row.split('_')[-1]
        elif '/' in row:
            return row.split('/')[-1]
        elif '미니-' in row:
            row = row.replace('미니-', '')
            return row
        elif ' ' in row and '(' not in row:
            return row.split()[-1]
        elif ',' in row:
            return row.split(',')[-1].strip()
        elif row == '블':
            return '블루'
        else:
            return row
    
review['컬러'] = review['컬러'].apply(color)


# 타입 컬럼
def type_prc(row):
    if not row: # None값 제외
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif ' [예약판매]' in row:
        row = row.replace(' [예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif row[0].isdigit() and ' ' not in row: # return T/F
        return None
    elif row[0].isdigit() and ' ' in row:
        return row.split('-')[-1]
    elif '(' not in row: # 타입과 무관한값은 ()문자가 없음
        return None
    else:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

# review['데님'] = review['데님'].apply(denim_color)
# review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
# review['타입2'] = review['타입2'].apply(denim_color)

In [11]:
# 카디널리티 확인
cols = ['타입', '사이즈','size', '컬러', '색상', '색상-사이즈'] #, '색상-사이즈', '데님'
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

타입
[None '크림(cream)-F' '라일락그레이(L.grey)-F' '블랙(black)-F' '핑크(pink)-F'
 '네이비(navy)-F' '화이트(white)-F' '그린(green)-F' '블랙(black)-3' '블랙(black)-2'
 '아이보리(ivory)-1' '블랙(black)-1' '아이보리(ivory)-2' '아이보리(ivory)-3'
 '화이트(white)-2' '라이트옐로우(L.yellow)-3' '화이트(white)-3' '화이트(white)-1'
 '베이지(beige)-2' '라이트옐로우(L.yellow)-2' '베이지(beige)-3' '라이트옐로우(L.yellow)-1'
 '베이지(beige)-1' '퓨어베이지(P.beige)-3' '핑크(pink)-3' '퓨어베이지(P.beige)-1'
 '퓨어베이지(P.beige)-2' '핑크(pink)-2' '모카(mocha)-1' '모카(mocha)-3' '모카(mocha)-2'
 '라벤더핑크(pink)-3' '뮤트블루(blue)-2' '뮤트블루(blue)-3' '라벤더핑크(pink)-2'
 '라벤더핑크(pink)-1' '차콜(charcoal)-1' '차콜(charcoal)-2' '뮤트핑크(M.pink)-2'
 '뮤트핑크(M.pink)-1' '크림(cream)-1' '뮤트블루(blue)-1' '뮤트핑크(M.pink)-3'
 '크림(cream)-3' '크림(cream)-2' '코발트블루(cobalt blue)-2' '코발트블루(cobalt blue)-1'
 '라벤더(lavender)-2' '라벤더(lavender)-3' '라벤더(lavender)-1'
 '코발트블루(cobalt blue)-3' '스카이(sky)-2' '옐로우(yellow) - F' '블랙(black) - F'
 '그레이(grey) - F' '핑크(pink)-1' '카키(khaki)-3' '라이트블루(L.blue)-2'
 '라이트블루(L.blue)-1' '차콜(charcoal)-3' '연그레이(l.grey)-3' '민트

In [52]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   타입      17278 non-null  object
 7   리뷰      17996 non-null  object
 8   사이즈     273 non-null    object
 9   size    5 non-null      object
 10  컬러      65 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      38 non-null     object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


In [53]:
size_idx = review.loc[review['size'].notnull()==True].index
for i in size_idx:
    review.loc[i, '사이즈'] = '1'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   타입      17278 non-null  object
 7   리뷰      17996 non-null  object
 8   사이즈     278 non-null    object
 9   size    5 non-null      object
 10  컬러      65 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      38 non-null     object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


In [54]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.loc[i, col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)[0]
        # print(row)
    return df

# 사이즈 컬럼 별도 수행
s_idx = review.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
for s in s_idx:
    review.loc[s, ['색상', '사이즈']] = review.iloc[s]['사이즈'].split('-')
        
cols = ['타입', '컬러', '색상-사이즈'] # , '색상-사이즈', '데님'
review = col_merge(review, cols)
review.sample(5)

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,size,컬러,색상-사이즈,색상
11087,583,a806120,5,None,None,None,오트밀(oatmeal)-F,사이즈 넉넉하고 좋아요! 길이감도 괜찮아서 계속 손이 가네요~!! 완전 반팔이 아니...,F,None,None,None,오트밀
2239,577,eunyih,5,156~160,65~,3,블랙(black)-F,포인트되고 아주 좋아요 살랑살랑 티 덕분에 시원함! 사이즈가 넉넉해서 편하게 입을 ...,F,None,None,None,블랙
12859,589,177007957,5,161~165,51 ~ 55,1,화이트(white)-1,"속에 많이 비치지도 않고 청바지 , 슬랙스 바지에 휘뚜루마뚜루 코디하기 좋아요 !!!",1,None,None,None,화이트
12689,589,177818083,5,None,46 ~ 50,1,소라(L.teal)-2,이뻐요~캐쥬얼하게도 격식있게도 입을수 있어요,2,None,None,None,소라
16008,612,166225460,5,None,None,None,차콜(charcoal)-F,부들부들 옷감좋아요,F,None,None,None,차콜


In [55]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   타입      17278 non-null  object
 7   리뷰      17996 non-null  object
 8   사이즈     17731 non-null  object
 9   size    5 non-null      object
 10  컬러      65 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      17557 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


* 카디널리티 확인

In [25]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
[None 'F' '3' '1' '2' ' F' 'L' '4']


색상
[None '라일락그레이' '크림' '블랙' '네이비' '핑크' '화이트' '그린' '아이보리' '라이트옐로우' '베이지'
 '퓨어베이지' '모카' '라벤더핑크' '뮤트블루' '차콜' '뮤트핑크' '라벤더' '코발트블루' '스카이' '옐로우' '그레이'
 '카키' '라이트블루' '연그레이' '민트' '레드오렌지' '소라' '메란지' '데님' '라이트퍼플' '코랄' '매트브라운'
 '연그린' '라이트민트' '오트밀' '퍼플' '브라운' '오렌지' '옐로우네이비' '그린네이비' '백메란지' '카키브라운'
 '샌드베이지' '워터블루' '퓨어아이보리' '진청' '연청' '백오트밀' '썸머오렌지' '워터리블루' '오트베이지' '연핑크'
 '라이트오렌지' '진네이비' '크림아이보리' '딥블루' '백멜란지' '백염' '블루' '레드브라운' '머스타드' '딥민트'
 '카키그레이']




In [56]:
def sel_size(row):
    size = ['S', 'M', 'L', 'XL', 'XL']
    size_dict = {size[i-1]: str(i) for i in range(1,6)}
    if row:
        if row.isdigit() or row=='F':
            return row
        elif row == ' F':
            return row.strip()
        else:
            return size_dict[row]

review['사이즈'] = review['사이즈'].apply(sel_size)
review['사이즈'].unique()

array([None, 'F', '3', '1', '2', '4'], dtype=object)

In [57]:
# 현재까지 처리된 df 사본
review_prc = review.copy()

#### 사이즈 컬럼 추가 전처리

In [ ]:
# def last_size(row):
#     if row:
#         if row == 'L':
#             return '3'
#         elif row == 'B)':
#             return None
#         elif row == ' F':
#             return 'F'
#         else:
#             return row
# review_prc['사이즈'] = review_prc['사이즈'].apply(last_size)

# review_prc['사이즈'].unique()

            

* 나머지 컬럼은 삭제

In [58]:
review_prc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   타입      17278 non-null  object
 7   리뷰      17996 non-null  object
 8   사이즈     17731 non-null  object
 9   size    5 non-null      object
 10  컬러      65 non-null     object
 11  색상-사이즈  175 non-null    object
 12  색상      17557 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


In [59]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review_prc.drop(columns=['타입','size', '컬러', '색상-사이즈'], inplace=True)

### '별점' 컬럼 핫핑 직원 데이터는 제외

In [37]:
print(review_prc['아이디'].unique())

['핫핑-연재' '핫핑-민지' '150567241' ... '5558742' '108182229' '1941037']


In [60]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review_prc['아이디'].unique():
    r = p.search(str(name))
    if r:
        staffs.add(name)
        
for s in staffs:
    review_prc.loc[review_prc['아이디'] == s, '별점'] = -1 # 분석에 사용하지 않음
          

In [61]:
review_prc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      17996 non-null  int64 
 1   아이디     17996 non-null  object
 2   별점      17996 non-null  int64 
 3   키       13604 non-null  object
 4   몸무게     12569 non-null  object
 5   평소사이즈   12404 non-null  object
 6   리뷰      17996 non-null  object
 7   사이즈     17731 non-null  object
 8   색상      17557 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.2+ MB


In [62]:
review_prc['별점'].value_counts()

별점
 5    16428
 4      787
 3      406
-1      155
 2      130
 1       90
Name: count, dtype: int64

In [63]:
review_prc.to_json('../Load/prc_data/reviews_prc_3.json', orient='records', indent=2, force_ascii=False)